In [31]:
%cd /workspace

from pathlib import Path

import numpy as np
import pandas as pd
import polars as pl
from tqdm import tqdm

/workspace


In [44]:
tqdm.pandas()

In [50]:
INPUT = Path("/workspace/resources/input")
task_df = pd.read_parquet(INPUT / "task2_dataset_raw_train.parquet")
poi_df = pd.read_parquet(INPUT / "cell_POIcat.parquet")

# task_df = pd.concat([task_df] * 4)

In [51]:
task_df

,uid,d,t,x,y
0,2381,0,15,158,99
1,2381,0,16,167,90
2,2381,0,19,167,88
3,2381,0,20,167,88
4,2381,0,23,168,88
...,...,...,...,...,...
26621231,1792,74,42,75,135
26621232,1792,74,43,75,135
26621233,1792,74,45,75,135
26621234,1792,74,46,75,135


In [52]:
x = list(range(1, 201))
y = list(range(1, 201))
mesh_map_df = pd.DataFrame(
    {
        "x": np.repeat(x, len(x)),
        "y": np.tile(y, len(y)),
        "mesh_id": range(len(x) * len(y)),
    }
)

In [53]:
import numpy as np
from tqdm import tqdm


def get_neighbors(i, j, n):
    directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    neighbors = []
    for di, dj in directions:
        ni, nj = i + di, j + dj
        if 0 <= ni < n and 0 <= nj < n:
            neighbors.append(nj * n + ni)
    return neighbors


def generate_adjacency_matrix(n=200):
    adj_matrix = np.zeros((n * n, n * n), dtype=int)

    for i in tqdm(range(n)):
        for j in range(n):
            mesh_id = j * n + i
            for neighbor_id in get_neighbors(i, j, n):
                adj_matrix[mesh_id][neighbor_id] = 1

    return adj_matrix


def get_kth_adjacency(adj_matrix, k):
    if k >= 2:
        kth_adj = np.linalg.matrix_power(adj_matrix, k)
        kth_adj[kth_adj > 1] = 1
        return kth_adj
    return adj_matrix


# 使用例
n = 200
k = 1
adj_matrix = generate_adjacency_matrix(n)
adj_matrix_kth = get_kth_adjacency(adj_matrix, k)
print(np.nonzero(adj_matrix_kth[0])[0])  # 0番のIDの隣接IDを表示
adj_matrix

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 520.54it/s]

[  1 200 201]


array([[0, 1, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 1, 0]])

In [58]:
adj_matrix.sum(axis=1).max()

8

In [180]:
def get_neighbor_mesh_id(mesh_id, adj_matrix, padding_value=-1, max_neighbor_num=8):
    neighbor_mesh_ids = np.nonzero(adj_matrix[mesh_id])[0].tolist()
    if (d := max_neighbor_num - len(neighbor_mesh_ids)) > 0:
        return neighbor_mesh_ids + [padding_value] * d
    return neighbor_mesh_ids


def make_mesh_map_df(adj_matrix, padding_value=-1):
    x = list(range(1, 201))
    y = list(range(1, 201))
    mesh_map_df = pd.DataFrame(
        {
            "x": np.repeat(x, len(x)),
            "y": np.tile(y, len(y)),
            "mesh_id": range(len(x) * len(y)),
        }
    )
    max_neighbor_num = adj_matrix.sum(axis=1).max()
    mesh_map_df["neighbor_mesh_ids"] = mesh_map_df["mesh_id"].progress_apply(
        lambda m: get_neighbor_mesh_id(
            m,
            adj_matrix,
            padding_value=padding_value,
            max_neighbor_num=max_neighbor_num,
        )
    )
    return mesh_map_df


mesh_map_df = make_mesh_map_df(adj_matrix)
mesh_map_df

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40000/40000 [00:07<00:00, 5315.25it/s]


,x,y,mesh_id,neighbor_mesh_ids
0,1,1,0,"[1, 200, 201, -1, -1, -1, -1, -1]"
1,1,2,1,"[0, 2, 200, 201, 202, -1, -1, -1]"
2,1,3,2,"[1, 3, 201, 202, 203, -1, -1, -1]"
3,1,4,3,"[2, 4, 202, 203, 204, -1, -1, -1]"
4,1,5,4,"[3, 5, 203, 204, 205, -1, -1, -1]"
...,...,...,...,...
39995,200,196,39995,"[39794, 39795, 39796, 39994, 39996, -1, -1, -1]"
39996,200,197,39996,"[39795, 39796, 39797, 39995, 39997, -1, -1, -1]"
39997,200,198,39997,"[39796, 39797, 39798, 39996, 39998, -1, -1, -1]"
39998,200,199,39998,"[39797, 39798, 39799, 39997, 39999, -1, -1, -1]"


In [162]:
def get_neighbor_mesh_id(mesh_id, adj_matrix, padding_value=-1, max_neighbor_num=8):
    neighbor_mesh_ids = np.nonzero(adj_matrix[mesh_id])[0].tolist()
    if (d := max_neighbor_num - len(neighbor_mesh_ids)) > 0:
        return neighbor_mesh_ids + [padding_value] * d
    return neighbor_mesh_ids


mesh_map_df["neighbor_mesh_ids"] = mesh_map_df["mesh_id"].progress_apply(
    lambda m: get_neighbor_mesh_id(m, adj_matrix)
)
df = task_df.head(100000).copy()
mesh_map_df

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 40000/40000 [00:06<00:00, 5805.01it/s]


,x,y,mesh_id,neighbor_mesh_ids
0,1,1,0,"[1, 200, 201, -1, -1, -1, -1, -1]"
1,1,2,1,"[0, 2, 200, 201, 202, -1, -1, -1]"
2,1,3,2,"[1, 3, 201, 202, 203, -1, -1, -1]"
3,1,4,3,"[2, 4, 202, 203, 204, -1, -1, -1]"
4,1,5,4,"[3, 5, 203, 204, 205, -1, -1, -1]"
...,...,...,...,...
39995,200,196,39995,"[39794, 39795, 39796, 39994, 39996, -1, -1, -1]"
39996,200,197,39996,"[39795, 39796, 39797, 39995, 39997, -1, -1, -1]"
39997,200,198,39997,"[39796, 39797, 39798, 39996, 39998, -1, -1, -1]"
39998,200,199,39998,"[39797, 39798, 39799, 39997, 39999, -1, -1, -1]"


In [163]:
merged_df = pd.merge(df, mesh_map_df, how="left", on=["x", "y"])
merged_df

,uid,d,t,x,y,mesh_id,neighbor_mesh_ids
0,2381,0,15,158,99,31498,"[31297, 31298, 31299, 31497, 31499, 31697, 316..."
1,2381,0,16,167,90,33289,"[33088, 33089, 33090, 33288, 33290, 33488, 334..."
2,2381,0,19,167,88,33287,"[33086, 33087, 33088, 33286, 33288, 33486, 334..."
3,2381,0,20,167,88,33287,"[33086, 33087, 33088, 33286, 33288, 33486, 334..."
4,2381,0,23,168,88,33487,"[33286, 33287, 33288, 33486, 33488, 33686, 336..."
...,...,...,...,...,...,...,...
99995,18618,5,14,135,98,26897,"[26696, 26697, 26698, 26896, 26898, 27096, 270..."
99996,18618,5,15,139,88,27687,"[27486, 27487, 27488, 27686, 27688, 27886, 278..."
99997,18618,5,17,141,86,28085,"[27884, 27885, 27886, 28084, 28086, 28284, 282..."
99998,18618,5,21,141,86,28085,"[27884, 27885, 27886, 28084, 28086, 28284, 282..."


In [183]:
poi_cat_cnt_df

POIcat_01_count  POIcat_02_count  POIcat_03_count  POIcat_04_count  \
x   y                                                                         
1   1                0.0              0.0              0.0              0.0   
    2                0.0              0.0              0.0              0.0   
    3                0.0              0.0              0.0              0.0   
    4                0.0              0.0              0.0              0.0   
    5                0.0              0.0              0.0              0.0   
...                  ...              ...              ...              ...   
200 196              0.0              0.0              0.0              0.0   
    197              0.0              0.0              0.0              1.0   
    198              0.0              0.0              0.0              1.0   
    199              0.0              0.0              0.0              1.0   
    200              0.0              0.0              0.0              0.0   

         POIcat_05_count  POIcat_06_count  POIcat_07_count  POIcat_08_count  \
x   y                                                                         
1   1                0.0              0.0              0.0              0.0   
    2                0.0              0.0              0.0              0.0   
    3                0.0              0.0              0.0              0.0   
    4                0.0              0.0              0.0              0.0   
    5                0.0              0.0              0.0              0.0   
...                  ...              ...              ...              ...   
200 196              0.0              0.0              0.0              0.0   
    197              0.0              0.0              0.0              0.0   
    198              0.0              0.0              1.0              0.0   
    199              2.0              0.0              0.0              0.0   
    200              0.0              0.0              0.0              0.0   

         POIcat_09_count  POIcat_10_count  ...  POIcat_76_count  \
x   y                                      ...                    
1   1                0.0              0.0  ...              0.0   
    2                0.0              0.0  ...              0.0   
    3                0.0              0.0  ...              0.0   
    4                0.0              0.0  ...              0.0   
    5                0.0              0.0  ...              1.0   
...                  ...              ...  ...              ...   
200 196              0.0              0.0  ...              0.0   
    197              0.0              0.0  ...              1.0   
    198              0.0              0.0  ...              0.0   
    199              0.0              0.0  ...              0.0   
    200              0.0              0.0  ...              1.0   

         POIcat_77_count  POIcat_78_count  POIcat_79_count  POIcat_80_count  \
x   y                                                                         
1   1                0.0              0.0              2.0              0.0   
    2                0.0              0.0              0.0              1.0   
    3                0.0              0.0              0.0              0.0   
    4                0.0              0.0              0.0              0.0   
    5                0.0              0.0              0.0              0.0   
...                  ...              ...              ...              ...   
200 196              0.0              0.0              0.0              0.0   
    197              0.0              0.0              0.0              0.0   
    198              0.0              0.0              1.0              0.0   
    199              0.0              0.0              0.0              0.0   
    200              0.0              1.0              1.0              0.0   

         POIcat_81_count  POIcat_82_count  POIcat_83_

In [164]:
node_feature = (
    pd.merge(
        mesh_map_df.drop("neighbor_mesh_ids", axis=1),
        poi_cat_cnt_df.reset_index(),
        on=["x", "y"],
        how="left",
    )
    .drop(["x", "y", "mesh_id"], axis=1)
    .fillna(0)
    .astype(np.int16)
    .to_numpy()
)

In [165]:
grouped = merged_df.groupby("uid", sort=False)
sequences = [
    torch.tensor(
        node_feature[np.array(group["neighbor_mesh_ids"].tolist(), dtype=np.int32)]
    )
    for _, group in grouped
]  # list[(sequence_len, neighbor_node_num, node_feature)]

In [166]:
len(sequences)

167

In [167]:
s = sequences[0]
s.shape

torch.Size([571, 8, 85])

In [182]:
a = "f_sss"
"fn" + a[1:]

'fn_sss'

In [ ]:
def make_node_features(mesh_map_df, poi_cat_cnt_df):
    node_feature = (
        pd.merge(
            mesh_map_df.drop("neighbor_mesh_ids", axis=1),
            poi_cat_cnt_df.reset_index(),
            on=["x", "y"],
            how="left",
        )
        .drop(["x", "y", "mesh_id"], axis=1)
        .fillna(0)
        .astype(np.int16)
        .to_numpy()
    )
    return node_feature

In [ ]:
def make_neighbor_node_sequences(df, mesh_map_df, poi_cat_cnt_df, group_key="uid"):
    node_feature = (
        pd.merge(
            mesh_map_df.drop("neighbor_mesh_ids", axis=1),
            poi_cat_cnt_df.reset_index(),
            on=["x", "y"],
            how="left",
        )
        .drop(["x", "y", "mesh_id"], axis=1)
        .fillna(0)
        .astype(np.int16)
        .to_numpy()
    )
    grouped = df.groupby(group_key, sort=False)
    sequences = [
        torch.tensor(
            node_feature[np.array(group["neighbor_mesh_ids"].tolist(), dtype=np.int32)]
        )
        for _, group in grouped
    ]  # list[(sequence_len, neighbor_node_num, node_feature)]
    return sequences

In [168]:
from torch.nn.utils.rnn import pad_sequence


def pad_3d_sequences(sequences, padding_value=0.0):
    # 各シーケンスの長さを取得
    seq_lengths = [seq.size(0) for seq in sequences]

    # 全体の最大のsequence_lenを取得
    max_seq_len = max(seq_lengths)

    # パディング後のテンソルのサイズを取得
    batch_size, num_neighbor_node, feature_dim = (
        len(sequences),
        sequences[0].size(1),
        sequences[0].size(2),
    )

    # ゼロのテンソルを事前に作成
    padded_tensor = torch.full(
        (batch_size, max_seq_len, num_neighbor_node, feature_dim), padding_value
    )

    # 各シーケンスをゼロのテンソルにコピー
    for i, seq in enumerate(sequences):
        padded_tensor[i, : seq_lengths[i]] = seq

    return padded_tensor


padding_value = -1
neighbor_node_seqs = sequences[:200]
feature_seqs_padded = pad_3d_sequences(
    [(seq) for seq in neighbor_node_seqs],
    padding_value=padding_value,
)  # (sequence_len, feature_dim)

feature_seqs_padded.shape

torch.Size([167, 621, 8, 85])

In [156]:
feature_seqs_padded[1]

tensor([[[ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0],
         ...,
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  1],
         [ 0,  0,  0,  ...,  0,  0,  0]],

        [[ 0,  0,  0,  ...,  0,  0,  0],
         [ 1,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0],
         ...,
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0]],

        [[ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0],
         ...,
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  1,  0]],

        ...,

        [[-1, -1, -1,  ..., -1, -1, -1],
         [-1, -1, -1,  ..., -1, -1, -1],
         [-1, -1, -1,  ..., -1, -1, -1],
         ...,
         [-1, -1, -1,  ..., -1, -1, -1],
         [-1, -1, -1, 

In [170]:
import torch


def pad_3d_sequences(sequences, padding_value=0.0):
    # 各シーケンスの長さを取得
    seq_lengths = [seq.size(0) for seq in sequences]

    # 全体の最大のsequence_lenを取得
    max_seq_len = max(seq_lengths)

    # パディング後のテンソルのサイズを取得
    batch_size, num_neighbor_node, feature_dim = (
        len(sequences),
        sequences[0].size(1),
        sequences[0].size(2),
    )

    # ゼロのテンソルを事前に作成
    padded_tensor = torch.full(
        (batch_size, max_seq_len, num_neighbor_node, feature_dim), padding_value
    )

    # padding_maskの初期化（Trueで初期化し、後で必要な部分をFalseに設定）
    padding_mask = torch.zeros(
        batch_size, max_seq_len, num_neighbor_node, dtype=torch.bool
    )

    # 各シーケンスをゼロのテンソルにコピーと、padding_maskの更新
    for i, seq in enumerate(sequences):
        padded_tensor[i, : seq_lengths[i]] = seq
        padding_mask[i, : seq_lengths[i]] = True

    return padded_tensor, padding_mask


# 仮のデータ
neighbor_node_seqs = [torch.randn(5, 3, 4), torch.randn(6, 3, 4), torch.randn(7, 3, 4)]
padding_value = 0.0

feature_seqs_padded, mask = pad_3d_sequences(
    neighbor_node_seqs, padding_value=padding_value
)

print(
    feature_seqs_padded.shape
)  # (batch_size, sequence_len, num_neighbor_node, feature_dim)
print(mask.shape)  # (batch_size, sequence_len, num_neighbor_node)
print(mask)  # Trueの部分がパディングされている部分

torch.Size([3, 7, 3, 4])
torch.Size([3, 7, 3])
tensor([[[ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True],
         [False, False, False],
         [False, False, False]],

        [[ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True],
         [False, False, False]],

        [[ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True]]])


In [127]:
import numpy as np

# 仮のデータ
neighbor_node_mtx = np.array(
    [[[0, 1, 1], [2, 3, 1]], [[1, 2, 1], [0, 3, 1]], [[1, 2, 1], [0, 3, 1]]]
)  # shape: (2, 2, 2)
node_feature = np.array(
    [[0.1, 0.2], [0.3, 0.4], [0.5, 0.6], [0.7, 0.8]]
)  # shape: (4, 2)

result = node_feature[neighbor_node_mtx]

print(result.shape)

(3, 2, 3, 2)


In [6]:
class GroupedSimpleFeatureExtoractor:
    def __init__(self, group_key, group_values, agg_methods):
        self.group_key = group_key
        self.group_values = group_values
        self.agg_methods = agg_methods

        self.group_key_name = "_".join(group_key)

    def __call__(self, df):
        agg_df = df.groupby(self.group_key)[self.group_values].agg(self.agg_methods)
        agg_df.columns = [
            f"{x[0]}_grpby_{self.group_key_name}_agg_{x[1]}" for x in agg_df.columns
        ]
        return (
            pd.merge(
                df[self.group_key],
                agg_df,
                how="left",
                left_on=self.group_key,
                right_index=True,
            )
            .drop(self.group_key, axis=1)
            .add_prefix("f_")
        )


df = task_df.copy()
group_key = ["uid"]
group_values = ["x", "y"]
agg_methods = ["mean", "max"]

e = GroupedSimpleFeatureExtoractor(group_key, group_values, agg_methods)
e(df)

,f_x_grpby_uid_agg_mean,f_x_grpby_uid_agg_max,f_y_grpby_uid_agg_mean,f_y_grpby_uid_agg_max
0,152.626970,190,94.427320,199
1,152.626970,190,94.427320,199
2,152.626970,190,94.427320,199
3,152.626970,190,94.427320,199
4,152.626970,190,94.427320,199
...,...,...,...,...
26621231,75.556635,93,131.611904,171
26621232,75.556635,93,131.611904,171
26621233,75.556635,93,131.611904,171
26621234,75.556635,93,131.611904,171


In [35]:
class TimeGroupedSimpleFeatureExtoractor:
    def __init__(self, group_key, group_values, time_range, agg_methods):
        self.group_key = group_key
        self.group_values = group_values
        self.time_range = time_range
        self.agg_methods = agg_methods

        self.group_key_name = "_".join(group_key)

        self.d_range = list(range(*time_range["d"])) if "d" in self.time_range else None
        self.t_range = list(range(*time_range["t"])) if "t" in self.time_range else None

        self.time_range_name = self.format_dict(time_range)

    @staticmethod
    def format_dict(d):
        result = []
        for key, values in d.items():
            result.append(f"{key}{values[0]}_{values[1]}")
        return "_".join(result)

    def __call__(self, df):
        selected_df = (
            df[df["d"].isin(self.d_range)].reset_index(drop=True)
            if self.d_range is not None
            else df.copy()
        )
        selected_df = (
            df[df["t"].isin(self.t_range)].reset_index(drop=True)
            if self.t_range is not None
            else selected_df
        )

        agg_df = selected_df.groupby(self.group_key)[self.group_values].agg(
            self.agg_methods
        )
        agg_df.columns = [
            f"{x[0]}_grpby_{self.group_key_name}_agg_{x[1]}_{self.time_range_name}"
            for x in agg_df.columns
        ]
        return (
            pd.merge(
                df[self.group_key],
                agg_df,
                how="left",
                left_on=self.group_key,
                right_index=True,
            )
            .drop(self.group_key, axis=1)
            .add_prefix("f_")
        )


group_key = ["uid"]
group_values = ["x", "y"]
time_range = {"d": [0, 7], "t": [0, 30]}
agg_methods = ["mean", "max"]

e = TimeGroupedSimpleFeatureExtoractor(group_key, group_values, time_range, agg_methods)
e(df)

,f_x_grpby_uid_agg_mean_d0_7_t0_30,f_x_grpby_uid_agg_max_d0_7_t0_30,f_y_grpby_uid_agg_mean_d0_7_t0_30,f_y_grpby_uid_agg_max_d0_7_t0_30
0,152.807843,190,92.968627,199
1,152.807843,190,92.968627,199
2,152.807843,190,92.968627,199
3,152.807843,190,92.968627,199
4,152.807843,190,92.968627,199
...,...,...,...,...
99995,134.755556,141,94.533333,107
99996,134.755556,141,94.533333,107
99997,134.755556,141,94.533333,107
99998,134.755556,141,94.533333,107


### GCN Examples

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# Graph Convolution 層の定義
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)

    def forward(self, input, adj):
        support = torch.matmul(input, self.weight)
        output = torch.matmul(adj, support)
        return output


# Mesh GCN モデルの定義
class MeshGCN(nn.Module):
    def __init__(self, feature_dim, hidden_dim, num_classes):
        super(MeshGCN, self).__init__()
        self.gc1 = GraphConvolution(feature_dim, hidden_dim)
        self.gc2 = GraphConvolution(hidden_dim, num_classes)

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = self.gc2(x, adj)
        return x


# メッシュの特徴データと隣接行列の準備
mesh_features = torch.rand((4, 10))
adjacency_matrix = torch.tensor(
    [
        [0, 1, 0, 0],
        [1, 0, 1, 0],
        [0, 1, 0, 1],
        [0, 0, 1, 0],
    ],
    dtype=torch.float32,
)

# ユーザーのメッシュ位置のインデックス
user_indices = torch.tensor([0, 2, 3])  # user1はm1, user2はm3, user3はm4に位置

# ユーザーデータの取得
user_data = mesh_features[user_indices]  # features

# ユーザーに対応する隣接行列の部分の取得
user_adj = adjacency_matrix[user_indices][:, user_indices]

# モデルのインスタンス化と予測の実行
model = MeshGCN(10, 64, 1)
updated_features = model(user_data, user_adj)

print(updated_features)

tensor([[0.0000],
        [0.2945],
        [0.5518]], grad_fn=<MmBackward0>)


In [16]:
mesh_features.shape

torch.Size([4, 10])

In [17]:
user_indices

tensor([0, 2, 3])

In [18]:
user_data.shape

torch.Size([3, 10])

In [19]:
user_adj

tensor([[0., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.]])

In [20]:
updated_features.shape

torch.Size([3, 1])

### Transformer Example

In [74]:
import torch
import torch.nn as nn


class CustomTransformerModelV1(nn.Module):
    def __init__(
        self,
        input_size1,
        input_size2,
        d_model,
        output_size,
        nhead=8,
        num_encoder_layers=6,
        num_decoder_layers=6,
    ):
        super().__init__()
        self.embedding_src = nn.Linear(input_size1, d_model)
        self.embedding_tgt = nn.Linear(input_size2, d_model)

        self.transformer = nn.Transformer(
            d_model,
            nhead,
            num_encoder_layers,
            num_decoder_layers,
            batch_first=True,
        )
        self.out = nn.Linear(d_model, output_size)

    def forward(self, batch):
        x_src = self.embedding_src(batch["feature_seqs"])
        x_tgt = self.embedding_tgt(batch["auxiliary_seqs"])

        src_mask = batch["feature_padding_mask"]
        tgt_mask = batch["auxiliary_padding_mask"]

        x = self.transformer(
            src=x_src,
            tgt=x_tgt,
            src_key_padding_mask=src_mask,
            tgt_key_padding_mask=tgt_mask,
        )
        x = self.out(x)

        return x


# ハイパーパラメータ
batch_size = 32
input_size1 = 5
input_size2 = 12
d_model = 128
seq_len_src = 20
seq_len_tgt = 10
output_size = 1
nhead = 2

# モデルのインスタンス化
model = CustomTransformerModelV1(
    nhead=nhead,
    input_size1=input_size1,
    input_size2=input_size2,
    d_model=d_model,
    output_size=output_size,
)

# 入力データの準備
batch = {
    "feature_seqs": torch.randn(batch_size, seq_len_src, input_size1),
    "auxiliary_seqs": torch.randn(batch_size, seq_len_tgt, input_size2),
    "feature_padding_mask": (torch.randn(batch_size, seq_len_src) > 0.5),  # 仮のマスク
    "auxiliary_padding_mask": (torch.randn(batch_size, seq_len_tgt) > 0.5),  # 仮のマスク
}
# モデルの実行
output = model(batch)
print(output.shape)  # [batch_size, seq_len_tgt, output_size] の形になるはず

torch.Size([32, 10, 1])


In [25]:
import torch
import torch.nn as nn


class SequenceGNN(nn.Module):
    def __init__(self, node_feature_dim, hidden_dim):
        super(SequenceGNN, self).__init__()

        # Transformation for node features and neighbor node features
        self.node_linear = nn.Linear(node_feature_dim, hidden_dim)
        self.neighbor_linear = nn.Linear(node_feature_dim, hidden_dim)
        self.aggregate_linear = nn.Linear(hidden_dim * 2, node_feature_dim)

    def forward(self, node_features, neighbor_node_features):
        batch_size, sequence_len, _ = node_features.shape

        updated_sequences = []
        for t in range(sequence_len):
            node_feature_t = node_features[:, t, :]
            neighbor_node_feature_t = neighbor_node_features[:, t, :, :]

            # Transform node features
            transformed_node = self.node_linear(node_feature_t)

            # Aggregate neighbor node features
            transformed_neighbors = self.neighbor_linear(neighbor_node_feature_t)
            aggregated_neighbors = transformed_neighbors.mean(dim=1)  # Mean aggregation

            # Concatenate transformed node features with aggregated neighbor features
            concatenated = torch.cat([transformed_node, aggregated_neighbors], dim=-1)

            # Final transformation
            out = self.aggregate_linear(concatenated)
            updated_sequences.append(out)

        # Stack sequences
        updated_sequences = torch.stack(updated_sequences, dim=1)

        return updated_sequences


# Example
batch_size = 32
sequence_len = 10
node_feature_dim = 64
hidden_dim = 128
neighbor_node_num = 5

node_features = torch.rand((batch_size, sequence_len, node_feature_dim))
neighbor_node_features = torch.rand(
    (batch_size, sequence_len, neighbor_node_num, node_feature_dim)
)

gnn = SequenceGNN(node_feature_dim, hidden_dim)
updated_node_features = gnn(node_features, neighbor_node_features)

In [26]:
updated_node_features.shape

torch.Size([32, 10, 64])

In [177]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# GraphSAGE with RNN-like update mechanism
class GraphSAGE_RNN_Cell(nn.Module):
    def __init__(self, in_features, hidden_features, embd_features=7):
        super().__init__()

        self.central_node_embedding = nn.Linear(in_features, embd_features)
        self.aggregated_neighbors_embedding = nn.Linear(in_features, embd_features)
        self.combine_linear = nn.Linear(
            embd_features + hidden_features, hidden_features
        )

        self.update_gate = nn.Linear(embd_features + hidden_features, hidden_features)
        self.reset_gate = nn.Linear(embd_features + hidden_features, hidden_features)

    def forward(
        self, central_node_features, neighbor_node_features, mask, hidden_state
    ):
        # aggregated_neighbors shape: (batch_size, in_features)
        aggregated_neighbors = (neighbor_node_features * mask.unsqueeze(-1)).sum(
            dim=1
        ) / mask.sum(dim=1, keepdim=True).clamp(min=1)

        central_node_features = self.central_node_embedding(central_node_features)
        aggregated_neighbors = self.aggregated_neighbors_embedding(aggregated_neighbors)

        combined_features = central_node_features + aggregated_neighbors
        combined = torch.cat(
            [combined_features, hidden_state], dim=1
        )  # concat along feature dimension

        z = torch.sigmoid(self.update_gate(combined))
        r = torch.sigmoid(self.reset_gate(combined))

        combined_reset = torch.cat([central_node_features, r * hidden_state], dim=1)

        h_tilda = F.relu(self.combine_linear(combined_reset))

        new_hidden = (1 - z) * hidden_state + z * h_tilda

        return new_hidden  # shape: (batch_size, hidden_features)


class ExtendedModelV2(nn.Module):
    def __init__(
        self,
        in_features_sage,
        hidden_features_sage,
        input_size1_lstm,
        input_size2_lstm,
        hidden_size_lstm,
        output_size,
    ):
        super(ExtendedModelV2, self).__init__()

        self.graphsage_cell = GraphSAGE_RNN_Cell(in_features_sage, hidden_features_sage)
        self.hidden_features_sage = hidden_features_sage
        self.hidden_size_lstm = hidden_size_lstm

        self.lstm_cell_1 = nn.LSTMCell(
            hidden_features_sage + input_size1_lstm, hidden_size_lstm
        )
        self.lstm2 = nn.LSTM(
            input_size2_lstm, hidden_size_lstm, batch_first=True, bidirectional=True
        )
        self.out = nn.Linear(hidden_size_lstm * 2, output_size)

    def forward(
        self,
        central_node_features,
        neighbor_node_features,
        mask,
        feature_seqs,
        auxiliary_seqs,
    ):
        batch_size, sequence_len, _ = central_node_features.shape
        hidden_features_sage = self.hidden_features_sage
        hidden_features_lstm = self.hidden_size_lstm

        h_sage = torch.zeros(
            batch_size, hidden_features_sage, device=central_node_features.device
        )
        h_lstm1 = torch.zeros(
            batch_size, hidden_features_lstm, device=central_node_features.device
        )
        c_lstm1 = torch.zeros(
            batch_size, hidden_features_lstm, device=central_node_features.device
        )

        for i in range(sequence_len):
            h_sage = self.graphsage_cell(
                central_node_features[:, i, :],
                neighbor_node_features[:, i, :, :],
                mask[:, i, :],
                h_sage,
            )  # shape: (batch_size, hidden_features_sage)

            combined_input = torch.cat(
                [h_sage, feature_seqs[:, i, :]], dim=1
            )  # shape: (batch_size, hidden_features_sage + input_size1_lstm)
            h_lstm1, c_lstm1 = self.lstm_cell_1(combined_input, (h_lstm1, c_lstm1))

        h_lstm1 = h_lstm1.repeat(2, 1, 1)  # to input bidirectional
        c_lstm1 = c_lstm1.repeat(2, 1, 1)
        lstm2_outputs, _ = self.lstm2(
            auxiliary_seqs, (h_lstm1, c_lstm1)
        )  # shape: (batch_size, sequence_len, hidden_size_lstm * 2)
        outputs = self.out(
            lstm2_outputs
        )  # shape: (batch_size, sequence_len, output_size)

        return outputs


# Dummy input for the extended model
batch_size = 8
sequence_len = 5
in_features = 16
hidden_features = 32
max_num_neighbors = 4

central_node_features = torch.rand(
    batch_size, sequence_len, in_features
)  # shape: (batch_size, sequence_len, in_features)
neighbor_node_features = torch.rand(
    batch_size, sequence_len, max_num_neighbors, in_features
)  # shape: (batch_size, sequence_len, max_num_neighbors, in_features)
mask = torch.randint(
    0, 2, (batch_size, sequence_len, max_num_neighbors)
).float()  # shape: (batch_size, sequence_len, max_num_neighbors)
feature_seqs = torch.rand(
    batch_size, sequence_len, in_features * 2
)  # shape: (batch_size, sequence_len, in_features * 2)
auxiliary_seqs = torch.rand(
    batch_size, sequence_len * 2, in_features
)  # shape: (batch_size, sequence_len, in_features)

model = ExtendedModelV2(
    in_features_sage=in_features,
    hidden_features_sage=hidden_features,
    input_size1_lstm=in_features * 2,
    input_size2_lstm=in_features,
    hidden_size_lstm=hidden_features,
    output_size=2,
)
outputs = model(
    central_node_features, neighbor_node_features, mask, feature_seqs, auxiliary_seqs
)
print(outputs.shape)  # Expected output: (batch_size, sequence_len, in_features)

torch.Size([8, 10, 2])
